In [35]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

Будем анализировать MOOC-курсы. В качестве зависимой переменной будем использовать долю учащихся, успешно завершивших начатый курс, от общего количества поступивших на курс. А в качестве независимой переменной будем использовать совокупную трудоемкость курса в часах, которая определяется как произвдение продолжительности курса в неделях и недельной нагрузки в часах.

In [10]:
n_samples = 1000

Создадим переменную: продолжительность курса в неделях.

In [13]:
weeks = np.random.choice(np.arange(4,13), n_samples)

Создадим переменную: недельная нагрузка в часах.

In [14]:
hours_per_week = np.random.choice(np.arange(4,17), n_samples)

Создадим переменную: количество преподавателей на курсе.

In [18]:
teachers = np.random.choice(np.arange(1,6), n_samples)

Будем считать, что доля успешно завершивших курс обратно пропорциональна общей трудоемкости курса.  
Кроме того, добавим случайную компоненту.

Создадим "шум".

In [24]:
noise = np.random.choice(np.arange(1, 101)/(100*192), n_samples) 

Минимальная трудоемкость составляет 16 часов. Будем считать, что при такой трудоемкости доля студентов, завершивших курс, будет максимальная.

Успешное завершение курса - есть обратная величина трудоемкости.

In [26]:
success = (16 / (weeks * hours_per_week)) - noise

Проверим диапозон переменной success: он должен быть, с одной стороны, не больше 1, с другой стороны - не меньше 0.

In [29]:
success.min()

0.07828125

In [30]:
success.max()

0.99984375

Создадим dataframe.

In [31]:
data = pd.DataFrame({'weeks': weeks, 
                     'hours_per_week': hours_per_week, 
                     'teachers': teachers, 
                     'success': success})

In [34]:
data.head()

,weeks,hours_per_week,teachers,success
0,10,7,3,0.224874
1,8,7,2,0.282693
2,9,12,5,0.144919
3,5,5,3,0.639844
4,10,16,3,0.096875


Построим модель с 3 переменными (которая не будет работать).

In [36]:
x = data[['weeks', 'hours_per_week', 'teachers']]
y = data['success']

In [42]:
reg = LinearRegression().fit(x, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

y_pred = reg.predict(data[['weeks', 'hours_per_week', 'teachers']])
print('Error: {}'.format(mean_absolute_error(y_pred, y)))

Weights: [-0.03829044 -0.03133246 -0.00050473]
Bias: 0.8883191852489006
Error: 0.05250071308771926


Не люблю mean absolute error, поэтому вычислим mean percentage absolute error (MAPE). Использование такой метрики оправдано, потому что ранее мы убедились в отсутствии нулевых значений в выборке.

In [38]:
def mape_vectorized(y_test, y_pred): 
    return (np.abs(y_test - y_pred) / y_test * 100)[y_test != 0].mean()

In [44]:
MAPE = mape_vectorized(y, y_pred)
round(MAPE, 2)

25.08

Так себе модель...  ¯\\_(ツ)_/¯

Создадим новую переменную: простота освоения курса.

In [47]:
data['simplicity'] = 1 / (data['weeks'] * data['hours_per_week'])
data.head()

,weeks,hours_per_week,teachers,success,simplicity
0,10,7,3,0.224874,0.014286
1,8,7,2,0.282693,0.017857
2,9,12,5,0.144919,0.009259
3,5,5,3,0.639844,0.040000
4,10,16,3,0.096875,0.006250


Построим новую модель.

In [55]:
x = data[['simplicity']]
y = data['success']

In [56]:
reg = LinearRegression().fit(x, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

y_pred = reg.predict(data[['simplicity']])
print('Error: {}'.format(mean_absolute_error(y_pred, y)))

Weights: [16.00133149]
Bias: -0.002537507409667461
Error: 0.0012612005597462685


In [57]:
MAPE = mape_vectorized(y, y_pred)
round(MAPE, 2)

0.64

Если бы не было "шума", то MAPE был бы равен 0. Но без "шума" - это не честно)